In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv('/content/Fertilizer Prediction.csv')

In [ ]:
df

,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,26,52,38,Sandy,Maize,37,0,0,Urea
1,29,52,45,Loamy,Sugarcane,12,0,36,DAP
2,34,65,62,Black,Cotton,7,9,30,14-35-14
3,32,62,34,Red,Tobacco,22,0,20,28-28
4,28,54,46,Clayey,Paddy,35,0,0,Urea
...,...,...,...,...,...,...,...,...,...
94,25,50,32,Clayey,Pulses,24,0,19,28-28
95,30,60,27,Red,Tobacco,4,17,17,10-26-26
96,38,72,51,Loamy,Wheat,39,0,0,Urea
97,36,60,43,Sandy,Millets,15,0,41,DAP


In [ ]:
df['Fertilizer Name'].value_counts()

Urea        22
DAP         18
28-28       17
14-35-14    14
20-20       14
17-17-17     7
10-26-26     7
Name: Fertilizer Name, dtype: int64

In [ ]:
X = df.drop('Fertilizer Name', axis=1)
Y = df['Fertilizer Name']

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Temparature  99 non-null     int64 
 1   Humidity     99 non-null     int64 
 2   Moisture     99 non-null     int64 
 3   Soil Type    99 non-null     object
 4   Crop Type    99 non-null     object
 5   Nitrogen     99 non-null     int64 
 6   Potassium    99 non-null     int64 
 7   Phosphorous  99 non-null     int64 
dtypes: int64(6), object(2)
memory usage: 6.3+ KB


In [ ]:
Y

0         Urea
1          DAP
2     14-35-14
3        28-28
4         Urea
        ...   
94       28-28
95    10-26-26
96        Urea
97         DAP
98       20-20
Name: Fertilizer Name, Length: 99, dtype: object

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.3, random_state = 42)

In [ ]:
X_train.shape

(69, 8)

In [ ]:
X_test.shape

(30, 8)

In [ ]:
ohe = OneHotEncoder(drop='first')
scaler = StandardScaler()
X_train.head()

,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous
94,25,50,32,Clayey,Pulses,24,0,19
30,29,58,40,Red,Ground Nuts,24,0,23
28,37,70,32,Black,Oil seeds,12,0,39
89,29,58,61,Loamy,Cotton,11,0,38
5,26,52,35,Sandy,Barley,12,10,13


In [ ]:
preprocessor = ColumnTransformer(
    transformers = [
        ('onehotencoder', ohe, [3,4]),
        ('standardization', scaler, [0,1,2,5,6,7])
    ],
    remainder = 'passthrough'
)

In [ ]:
combined_X = pd.concat([X_train, X_test], axis=0)

# Define transformers
ohe = OneHotEncoder(drop='first')
scaler = StandardScaler()

# Define preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('onehotencoder', ohe, [3, 4]),
        ('standardization', scaler, [0, 1, 2, 5, 6, 7])
    ],
    remainder='passthrough'
)

# Fit and transform combined data
combined_X_dummy = preprocessor.fit_transform(combined_X)

# Split the transformed data back into training and testing sets
X_train_dummy = combined_X_dummy[:len(X_train)]
X_test_dummy = combined_X_dummy[len(X_train):]

# Display shapes of transformed data
print("X_train_dummy shape:", X_train_dummy.shape)
print("X_test_dummy shape:", X_test_dummy.shape)

X_train_dummy shape: (69, 20)
X_test_dummy shape: (30, 20)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=100, criterion='gini', random_state=42)
classifier.fit(X_train_dummy,Y_train)
pred = classifier.predict(X_test_dummy)
accuracy_score(Y_test, pred)


1.0

In [ ]:
def prediction(Temperature, Humidity, Moisture, SoilType, CropType, Nitrogen, Potassium, Phosphorous):
  features = np.array([[Temperature, Humidity, Moisture, SoilType, CropType, Nitrogen, Potassium, Phosphorous]])
  transformed_features = preprocessor.transform(features)
  predicted_Value = classifier.predict(transformed_features).reshape(1,-1)
  return predicted_Value[0]

In [ ]:
result = prediction(25,50,32,'Clayey','Pulses',24,0,19)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
result

array(['28-28'], dtype=object)

In [ ]:
import pickle

In [ ]:
filename = 'fertilizer_recommendation_trained.sav'
pickle.dump(classifier, open(filename, 'wb'))

In [ ]:
loaded_model = pickle.load(open('/content/fertilizer_recommendation_trained.sav', 'rb'))

In [ ]:
def prediction(Temperature, Humidity, Moisture, SoilType, CropType, Nitrogen, Potassium, Phosphorous):
  features = [[Temperature, Humidity, Moisture, SoilType, CropType, Nitrogen, Potassium, Phosphorous]]
  transformed_features = preprocessor.transform(features)
  predicted_Value = loaded_model.predict(transformed_features).reshape(1,-1)
  return predicted_Value[0]

In [ ]:
result = prediction(25,50,32,'Clayey','Pulses',24,0,19)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
result

array(['28-28'], dtype=object)

In [ ]:
filename = 'preprocessor.sav'
pickle.dump(preprocessor, open(filename, 'wb'))

In [ ]:
processor = pickle.load(open('/content/preprocessor.sav', 'rb'))

In [ ]:
def prediction(Temperature, Humidity, Moisture, SoilType, CropType, Nitrogen, Potassium, Phosphorous):
  features = np.array([[Temperature, Humidity, Moisture, SoilType, CropType, Nitrogen, Potassium, Phosphorous]])
  transformed_features = processor.transform(features)
  predicted_Value = loaded_model.predict(transformed_features).reshape(1,-1)
  return predicted_Value[0]

In [ ]:
result = prediction(25,50,32,'Clayey','Pulses',24,0,19)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
df['Soil Type'].unique()

array(['Sandy', 'Loamy', 'Black', 'Red', 'Clayey'], dtype=object)

In [ ]:
df['Crop Type'].unique()

array(['Maize', 'Sugarcane', 'Cotton', 'Tobacco', 'Paddy', 'Barley',
       'Wheat', 'Millets', 'Oil seeds', 'Pulses', 'Ground Nuts'],
      dtype=object)